In [1]:
# Importing libraries
import pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, row_number, max
from pyspark.sql.window import Window

In [2]:
# Creating spark enviroment
spark = pyspark.sql.SparkSession.builder.appName("CGE P6 T3").getOrCreate()

22/12/23 19:50:51 WARN Utils: Your hostname, martin resolves to a loopback address: 127.0.1.1; using 192.168.0.24 instead (on interface wlp1s0)
22/12/23 19:50:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/23 19:51:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Change this variable to decide where the dataset files will be obtained from
useRoot = False

# Dataset common path
common_path_root = "./"                         # Default dataset path
common_path_full = "../datasets/BX-CSV-Dump/"   # Custom dataset path for full data

# Dataset file names
users_name = "BX-Users.csv"
books_name = "BX-Books.csv"
ratings_name = "BX-Book-Ratings.csv"

# Dataset paths
path_full = (common_path_root if useRoot else common_path_full)
users_path = path_full + users_name
books_path = path_full + books_name
ratings_path = path_full + ratings_name

# Defining dataset schemas
user_schema = StructType([
    StructField("User-ID", IntegerType(), True),
    StructField("Location", StringType(), True),
    StructField("Age", IntegerType(), True)
])

book_schema = StructType([
    StructField("ISBN", StringType(), True),
    StructField("Book-Title", StringType(), True),
    StructField("Book-Author", StringType(), True),
    StructField("Year-Of-Publication", IntegerType(), True),
    StructField("Publisher", StringType(), True),
    StructField("Image-URL-S", StringType(), True),
    StructField("Image-URL-M", StringType(), True),
    StructField("Image-URL-L", StringType(), True)
])

rating_schema = StructType([
    StructField("User-ID", IntegerType(), True),
    StructField("ISBN", StringType(), True),
    StructField("Book-Rating", IntegerType(), True)
])

In [4]:
# Importing each dataset into a Dataframe
df_users = spark.read.csv(users_path, schema=user_schema, sep=";", header=True)
df_books = spark.read.csv(books_path, schema=book_schema, sep=";", header=True)
df_ratings = spark.read.csv(ratings_path, schema=rating_schema, sep=";", header=True)

# Removing key missing values
df_users = df_users.filter(col('User-ID').isNotNull()) # Removing users with null User-ID (one row found)

# Print one Datafram as a sample
df_users.show()

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|null|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|null|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|null|
|      6|santa monica, cal...|  61|
|      7| washington, dc, usa|null|
|      8|timmins, ontario,...|null|
|      9|germantown, tenne...|null|
|     10|albacete, wiscons...|  26|
|     11|melbourne, victor...|  14|
|     12|fort bragg, calif...|null|
|     13|barcelona, barcel...|  26|
|     14|mediapolis, iowa,...|null|
|     15|calgary, alberta,...|null|
|     16|albuquerque, new ...|null|
|     17|chesapeake, virgi...|null|
|     18|rio de janeiro, r...|  25|
|     19|           weston, ,|  14|
|     20|langhorne, pennsy...|  19|
+-------+--------------------+----+
only showing top 20 rows



In [5]:
# Showing list of users with the number of books they reviewed
# Grouping rating column by User-ID (for each user, one review can be submited per book, so each review is for a different book), and counting ratings
df_user_ratings_count = df_ratings.groupBy('User-ID').count().withColumnRenamed('count', 'Books-Reviewed')

# Adding review count to user table, removing location and age, and changing null count with 0
df_users.join(df_user_ratings_count, on='User-ID', how='left_outer').drop('Location', 'age').fillna({'Books-Reviewed': 0}).orderBy('User-ID').show()

+-------+--------------+
|User-ID|Books-Reviewed|
+-------+--------------+
|      1|             0|
|      2|             1|
|      3|             0|
|      4|             0|
|      5|             0|
|      6|             0|
|      7|             1|
|      8|            18|
|      9|             3|
|     10|             2|
|     11|             0|
|     12|             1|
|     13|             0|
|     14|             4|
|     15|             0|
|     16|             2|
|     17|             7|
|     18|             0|
|     19|             1|
|     20|             1|
+-------+--------------+
only showing top 20 rows



In [6]:
# Showing maximum rating received by each publisher
# Getting max rating for each book
df_book_ratings_max = df_ratings.groupBy('ISBN').max('Book-Rating')

# Joining with processed max rating for each book Dataframe, grouping by publisher,
# and then getting max rating of all max book ratings within the same publisher
df_books.join(df_book_ratings_max, on='ISBN', how='left_outer').groupBy('Publisher').max('max(Book-Rating)')\
    .withColumnRenamed('max(max(Book-Rating))', 'Max-Rating').show()

+--------------------+----------+
|           Publisher|Max-Rating|
+--------------------+----------+
|Harper Mass Marke...|        10|
|        Chosen Books|        10|
|Houghton Mifflin ...|        10|
|      Celestial Arts|        10|
|Ian Allan Publish...|         8|
|Crown Publishers ...|         0|
|Carroll &amp; Gra...|        10|
|Orion Business Books|         1|
|        Lorenz Books|        10|
|Phillimore &amp; ...|         7|
|Lucis Publishing ...|         6|
|Collectors' Infor...|         0|
| Research Press (IL)|         7|
|    Underwood-Miller|        10|
|    Daughters, inc.]|        10|
|        Book Pub. Co|         0|
|Chicago Review Press|        10|
|    Adams Media Corp|        10|
|         Hermetic Pr|        10|
|      Aqua Explorers|         9|
+--------------------+----------+
only showing top 20 rows



In [7]:
# Showing name of the author with the highest number of ratings
# Getting the number of ratings per book
df_book_ratings_count = df_ratings.groupBy('ISBN').count().withColumnRenamed('count', 'Ratings-Count')

# Joining book Dataframe with processed ratings count Dataframe, gouping by Author, and adding all rating counts per author's book
# Then, filling all null values with 0, and sorting by that number of ratings, and getting only first one
df_books.join(df_book_ratings_count, on='ISBN', how='left_outer').groupBy('Book-Author').sum('Ratings-Count')\
    .withColumnRenamed('sum(Ratings-Count)', 'Ratings-Count').fillna({'Ratings-Count': 0}).orderBy('Ratings-Count', ascending=0).limit(1).show()

+------------+-------------+
| Book-Author|Ratings-Count|
+------------+-------------+
|Stephen King|        10053|
+------------+-------------+



In [8]:
# Showing the name of the book with the highest number of ratings for each publisher
# Joining books and ratings Dataframes to add to books Dataframe the column with the ratings count,
# and dropping unnecessary columns
df_books_with_rating_count = df_books.join(df_book_ratings_count, on='ISBN', how='left_outer')\
    .drop('Book-Author', 'Year-Of-Publication', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'ISBN')

# Creating Window function that partitions the Dataframe by publisher, and sorts by ratings count in descending order
window_func = Window.partitionBy("Publisher").orderBy(df_books_with_rating_count['Ratings-Count'].desc())

# Adding column row number
df_books_with_rating_count = df_books_with_rating_count.withColumn("row_number", row_number().over(window_func))

# Get rows that have 1 as row number (for each publisher, the book with the most ratings), dropping row number column and filling null values with 0
df_books_with_rating_count.filter(df_books_with_rating_count["row_number"] == 1).drop("row_number").fillna({'Ratings-Count': 0}).show()

+--------------------+--------------------+-------------+
|          Book-Title|           Publisher|Ratings-Count|
+--------------------+--------------------+-------------+
|      The big chapel|(3 Queen Sq., WC1...|            1|
|Manna: foods of t...|101 Productions; ...|            2|
|        Life Drawing|     11/9 Publishing|            1|
| Portraits of Cities|22nd. Century, Ne...|            1|
|Restoring Intimac...|                   3|            1|
|300 Incredible Th...|   300Incredible.com|            4|
|Denver Hiking Gui...|            3D Press|            1|
|Haley's Cleaning ...|3H Productions, Inc.|            1|
|Stories in the Wo...|             3rd Bed|            3|
|              Dictee|     3rd Woman Press|            1|
|Heal Thyself for ...|A &amp; B Book Di...|            1|
|What's In Oscar's...|       A Golden Book|            3|
|The Subjection of...|  A H M Publications|            1|
|   Tears Of The Moon|A Jove Book, Berk...|            2|
|     Pent Up 

In [9]:
# Showing the difference between the number of ratings of each book and the number of ratings of the book with the most ratings within the same publisher
# Creating window fuction
window_func = Window.partitionBy('Publisher')

# Calculating the maximum number of ratings for each publisher
df_max = df_books_with_rating_count.withColumn('Max-Ratings-Count', max('Ratings-Count').over(window_func)).drop('row_number')

# Adding a new column with the difference between the number of ratings of each book and the maximum number of ratings for the same publisher
df_result = df_max.withColumn('Difference', df_max['Max-Ratings-Count'] - df_max['Ratings-Count'])\
    .orderBy('Publisher', 'Ratings-Count', ascending=False)
df_result.show()

+--------------------+--------------------+-------------+-----------------+----------+
|          Book-Title|           Publisher|Ratings-Count|Max-Ratings-Count|Difference|
+--------------------+--------------------+-------------+-----------------+----------+
|La Terre sous ses...|   �?�?ditions 10/18|            1|                1|         0|
|Atem und Stimme. ...|       �?¶bv&amp;hpt|            1|                1|         0|
|Maria Jeritza: Pr...|�?sterreichischer...|            1|                1|         0|
|Reiseführer ins J...|�?sterreichische ...|            1|                1|         0|
|Heimat los!: Aus ...|             �?lbaum|            1|                1|         0|
|Toi, ou personne:...|�?ditions la Matz...|            1|                1|         0|
|Fragments d'un di...|  �?ditions du Seuil|            2|                2|         0|
|Rue du prolétaire...|  �?ditions du Seuil|            1|                2|         1|
|La plus haute des...|  �?ditions du Seuil|